**Introduction to SQL**

Таблицы - маленькие буквы с нижними подчеркиваниями (можно и множественное число). Столбцы - нижний регистр и единственное число.

Типы данных:

* строки: тип VARCHAR - маленькие и большие строки

* целые числа: тип INT

* плавающий тип данных с запятой: NUMERIC 38 цифр до и после запятой

In [ ]:
SELECT name, year AS first_name, year_hired
FROM employees;

SELECT DISTINCT dept_id, year_hired # уникальные по двум столбцам
FROM employees;

CREATE VIEW employee_hire_years AS # сохранение запроса, потом через FROM юзаем
SELECT id, name, year_hired
FROM employees
LIMIT 2;

**Intermediate SQL**

In [ ]:
SELECT COUNT(birthate) AS count_birthdates # если COUNT(*) с пропусками
FROM people;
SELECT COUNT(DISTINCT) birthdate AS count_distinct_birthdates
FROM people;
# порядок FROM -> SELECT -> LIMIT
SELECT title, "release year", country # приходится делать так

In [ ]:
SELECT title
FROM films
WHERE release_year > 1960; # <> не равно, country = 'Japan'
# FROM -> WHERE -> SELECT -> LIMIT
SELECT titrle
FROM films
WHERE (release_year=1994 OR release_year = 1995)
AND (certification = 'PG' OR certification = 'R')

SELECT *
FROM coats
WHERE buttons BETWEEN 1 AND 5;

SELECT title
FROM films
WHERE release_year
BETWEEN 1994 AND 2000 AND country='UK';

In [ ]:
SELECT name
FROM people
WHERE name LIKE 'Ade%'; #'Ev_' только один символ, '__t%' два символа
WHERE name NOT LIKE 'A.%'; # '%r' пофиг на пробелы

SELECT title
FROM films
WHERE release_year IN (1920, 1930, 1940); # WHERE country IN ('Germany','France')

WHERE birthdate IS NULL; # WHERE birthdate IS NOT NULL;

In [ ]:
MIN(), MAX(), AVG(), SUM(), COUNT() #от A до Z
SELECT ROUND(AVG(budget), 0) AS avg_budget # можно и -5
FROM films
WHERE release_year >= 2010;

In [ ]:
# арифметика
SELECT (4 + 3); # |7|
SELECT (4 / 3); # |1|
SELECT (4.0 / 3.0); # |1.333...|
SELECT (gross - budget) AS profit # без имени поэтому псевдоним
FROM films;

SELECT title, budget
FROM films
WHERE budget IS NOT NULL
ORDER BY budget DESC; # убывание, можно ORDER BY wins DESC, imdb_score DESC;
# FROM -> WHERE -> SELECT -> ORDER BY -> LIMIT

In [ ]:
# группировка
SELECT certification, COUNT(title) AS title_count
FROM films
GROUP BY certification;

SELECT certification, language, COUNT(title) AS title_count
FROM films
GROUP BY certification, language;

# FROM -> GROUP BY -> ORDER BY
# В ORDER BY используем псевдонимы из SELECT

#практика-прикол
SELECT release_year, COUNT(DISTINCT language)
FROM films
GROUP BY release_year
ORDER BY count DESC; # хотя не т.к создался такой псевдоним

SELECT release_year, COUNT(title) AS title_count
FROM films
GROUP BY release_year
HAVING COUNT(title) > 10;
FROM -> WHERE -> GROUP BY -> HAVING -> SELECT -> ORDER BY -> LIMIT

**Joining Data in SQL**

In [ ]:
SELECT prime_ministers.country, prime_ministers.continent, prime_minister, president
FROM prime_ministers
INNER JOIN presidents
ON prime_ministers.country = presidents.country;

SELECT p1.country, p1.continent, prime_minister, president
FROM prime_ministers AS p1
INNER JOIN presidents AS p2
ON p1.country = p2.country;

INNER JOIN presidents AS p2
USING(country); # если общий столбец

SELECT *
FROM left_table
INNER JOIN right_table
ON left_table.id = right_table.id
INNER JOIN another_table
ON left_table.id = another_table.id;
# три типа внешних соединений
SELECT *
FROM left_table
INNER JOIN right_table
ON left_table.id = right_table.id AND left_table.date = right_table.date;

SELECT p1.country, prime_minister, president # левое соединение
FROM prime_ministers AS p1
LEFT JOIN presidents AS p2 # RIGHT JOIN right_table
USING(country);

SELECT left_table.id AS L_id, right_table.id AS R_id,
            left_table.val AS L_val, right_table.val AS R_val
FROM left_table
FULL JOIN right_table
USING (id);

In [ ]:
# cross join все со всеми
SELECT id1, id2
FROM table1
CROSS JOIN table2;

SELECT prime_minister, president
FROM prime_ministers AS p1
CROSS JOIN presidents AS p2
WHERE p1.continent IN ('Asia') AND p2.continent IN ('South America');

# self - join
SELECT p1.country AS country1, p2.country AS country2, p1.continent
FROM prime_ministers AS p1
INNER JOIN prime_ministers AS p2
  ON p1.continent = p2.continent AND p1.country <> p2.country; # плачь и смотри
LIMIT 10;

In [ ]:
# наложили друг на друга
SELECT monarch AS leader, country
FROM monarchs
UNION # без повторений # UNION ALL - тупо наложил
SELECT prime_minister, country
FROM prime_ministers
ORDER BY leader, country
LIMIT 10;

# пересечение
SELECT country, prime_minister AS leader # имена из первой таблицы
FROM prime_ministers # и имена не обязательно совпадают
INTERSECT
SELECT country, monarch
FROM monarchs;
# левое множество без пересечения и правой таблицы
SELECT monarch, country
FROM monarchs
EXCEPT
SELECT prime_minister, country
FROM prime_ministers;

In [ ]:
# semi join только те строки из левой где есть совпадение с правой
# позапрос внутри where
SELECT president, country, continent
FROM presidents
WHERE country IN
      (SELECT country
        FROM states
        WHERE indep_year < 1800);

SELECT country, president
FROM presidents
WHERE continent LIKE '%America'
  AND country NOT IN
      (SELECT country
        FROM states
        WHERE indep_year < 1800);
# подзапрос внутри WHERE
SELECT *
FROM some_table
WHERE some_field IN
    (SELECT some_numeric_field
    FROM another_table
    WHERE field2 = some_condition);
#подзапрос внутри SELECT
SELECT DISTINCT continent,
    (SELECT COUNT(*)
    FROM monarchs
    WHERE states.continent = monarch.continent) AS monarch_count
FROM states;
#подзапрос внутри FROM
SELECT DISTINCT monarchs.continent, sub.most_recent
FROM monarchs,
    (SELECT continent, MAX(indep_year) AS most_recent
      FROM states
      GROUP BY continent) AS sub
WHERE monarchs.continent = sub.continent
ORDER BY continent;

**Data Manipulation in SQL**

In [ ]:
# аналог операции ЕСЛИ это ТО то
CASE WHEN x = 1 THEN 'a'
     WHEN x = 2 THEN 'b'
     ELSE 'c' END AS new_column
# создание нового столбца
SELECT id, home_goal, away_goal,
      CASE WHEN home_goal > away_goal THEN 'Home Team Win'
      WHEN home_goal < away_goal THEN 'Away Team Win'
      ELSE 'Tie' END AS outcome # ELSE NULL END AS date_category
FROM match
WHERE season = '2013/2014';

# вот это пример (хотя понятно, конец IS NOT NULL)
SELECT date, season,
    CASE WHEN hometeam_id = 8455 AND home_goal > away_goal
          THEN 'Chelsea home win!'
        WHEN awayteam_id = 8455 AND home_goal < away_goal
          THEN 'Chelsea away win!' END AS outcome
FROM match
WHERE CASE WHEN hometeam_id = 8455 AND home_goal > away_goal
              THEN 'Chelsea home win!'
            WHEN awayteam_id = 8455 AND home_goal < away_goal
              THEN 'Chelsea away win!' END IS NOT NULL;

In [ ]:
SELECT season, COUNT(CASE WHEN hometeam_id = 8650
                          AND home_goal > away_goal
                          THEN id END) AS home_wins
FROM match
GROUP BY season;

SELECT season, SUM(CASE WHEN hometeam_id = 8650
                    THEN home_goal END) AS home_goals,
              SUM(CASE WHEN awayteam_id = 8650
                    THEN away_goal END) AS away_goals
FROM match
GROUP BY season;
# подсчет процентов
SELECT season,
       AVG(CASE WHEN hometeam_id = 8455 AND home_goal > away_goal THEN 1
                WHEN hometeam_id = 8455 AND home_goal < away_goal THEN 0
                END) AS pct_homewins,
       AVG(CASE WHEN awayteam_id = 8455 AND away_goal > home_goal THEN 1
                WHEN awayteam_id = 8455 AND away_goal < home_goal THEN 0
                END) AS pct_awaywins
FROM match
GROUP BY season;

Subqueries (подзапросы)

In [ ]:
# помещается SELECT , FROM , WHERE , GROUP BY
# простой подзапрос (выполняется один раз и сам по себе и перед основным запросом)
SELECT home_goal
FROM match
WHERE home_goal > (SELECT AVG(home_goal)
                    FROM match);
SELECT AVG(home_goal) FROM match;

SELECT team_long_name, team_short_name AS abbr
FROM team
WHERE team_api_id IN (SELECT hometeam_id
                      FROM match
                      WHERE country_id = 15722);

In [ ]:
SELECT team, home_avg
FROM (SELECT t.team_long_name AS team,
             AVG(m.home_goal) AS home_avg
      FROM match AS m
      LEFT JOIN team AS t
      ON m.hometeam_id = t.team_api_id
      WHERE season = '2011/2012'
      GROUP BY team) AS subquery # обязательно псевдоним
ORDER BY home_avg DESC
LIMIT 3;

In [ ]:
# позапрос в SELECT (создается столбец из одного числа)
SELECT season,
        COUNT(id) AS matches,
        (SELECT COUNT(id) FROM match) as total_matches
FROM match
GROUP BY season; # даже пофиг на это aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa

-- коммент для SQL

Коррелированные подзапросы

In [ ]:
# выполняются каждый раз когда поялвяется инфа из внешнего запроса
SELECT s.stage,
        ROUND(s.avg_goals,2) AS avg_goal,
        (SELECT AVG(home_goal + away_goal) FROM match
        WHERE season = '2012/2013') AS overall_avg
FROM
        (SELECT stage,
                AVG(home_goal + away_goal) AS avg_goals
         FROM match
         WHERE season = '2012/2013'
          GROUP BY stage) AS s
WHERE s.avg_goals > (SELECT AVG(home_goal + away_goal)
                     FROM match
                     WHERE season = '2012/2013');
# ну такое хз
SELECT s.stage,
       ROUND(s.avg_goals,2) AS avg_goal,
       (SELECT AVG(home_goal + away_goal)
        FROM match
        WHERE season = '2012/2013') AS overall_avg
FROM
    (SELECT stage,
        AVG(home_goal + away_goal) AS avg_goals
     FROM match
     WHERE season = '2012/2013'
     GROUP BY stage) AS s
WHERE s.avg_goals > (SELECT AVG(home_goal + away_goal)
                     FROM match AS m
                     WHERE s.stage > m.stage);
# второй пример (вместо join используем корр подзапрос)
SELECT c.name AS country,
       AVG(m.home_goal + m.away_goal) AS avg_goals
FROM country AS c
LEFT JOIN match AS m
ON c.id = m.country_id
GROUP BY country;

SELECT c.name AS country,
        (SELECT AVG(home_goal + away_goal)
                FROM match AS m
                WHERE m.country_id = c.id) AS avg_goals
FROM country AS c
GROUP BY country; #аааааааааааааааааааааааааааааааааааааааааааааа

In [ ]:
# вложенные позапросы (подзпросы в позапросах), могут быть корр и некорр
SELECT EXTRACT(MONTH FROM date) AS month,
       SUM(m.home_goal + m.away_goal) AS total_goals,
       SUM(m.home_goal + m.away_goal) -
       (SELECT AVG(goals)
        FROM (SELECT EXTRACT(MONTH FROM date) AS month,
              SUM(home_goal + away_goal) AS goals
              FROM match
              GROUP BY month)) AS avg_diff
FROM match AS m
GROUP BY month; # aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa

# пример когда вложенный подзапрос коррелирован
SELECT c.name AS country,
       (SELECT AVG(home_goal + away_goal)
        FROM match AS m
        WHERE m.country_id = c.id # Correlates with main query
              AND id IN (SELECT id # Begin inner subquery
                         FROM match
                         WHERE season = '2011/2012')) AS avg_goals
FROM country AS c
GROUP BY country; # почему тут country ааааааааааааааааааааааааааааааааааааааааааааааааааааааааааааа

CTE (общие табличные выражения)

In [ ]:
# это особый вид подзапроса
WITH s AS (SELECT country_id, id
           FROM match
           WHERE (home_goal + away_goal) >= 10)

SELECT c.name AS country,
       COUNT(s.id) AS matches
FROM country AS c
INNER JOIN s
ON c.id = s.country_id
GROUP BY country;

WITH s1 AS (SELECT country_id, id
            FROM match
            WHERE (home_goal + away_goal) >= 10),
     s2 AS (SELECT country_id, id
            FROM match
            WHERE (home_goal + away_goal) <= 1)
# CTE могут ссылаться на предыдущие и сами на себя (рекурсивный СТЕ)
SELECT c.name AS country,
       COUNT(s1.id) AS high_scores,
       COUNT(s2.id) AS low_scores
FROM country AS c
INNER JOIN s1
ON c.id = s1.country_id
INNER JOIN s2
ON c.id = s2.country_id
GROUP BY country;

Оконные функции

In [ ]:
# оплучаем статистические вычисления без группировки
SELECT date, (home_goal + away_goal) AS goals,
       (SELECT AVG(home_goal + away_goal)
        FROM match
        WHERE season = '2011/2012') AS overall_avg
FROM match
WHERE season = '2011/2012';
# можно проще через оконные фнукции
SELECT date,
       (home_goal + away_goal) AS goals,
        AVG(home_goal + away_goal) OVER() AS overall_avg
FROM match
WHERE season = '2011/2012';
|       date | goals |  overall_avg |
|------------|-------|--------------|
| 2011-07-29 |     3 |      2.71646 |
| 2011-07-30 |     2 |      2.71646 |

# ранжирование
SELECT date, (home_goal + away_goal) AS goals,
       RANK() OVER(ORDER BY home_goal + away_goal) AS goals_rank
FROM match
WHERE season = '2011/2012'; #низкие значение и 1 ранг, одинаковые значения = одинаковый ранг

SELECT date, (home_goal + away_goal) AS goals,
       RANK() OVER(ORDER BY home_goal + away_goal DESC) AS goals_rank
FROM match
WHERE season = '2011/2012'; # высокие зачения и 1 ранг

# с PARTITION
SELECT date, (home_goal + away_goal) AS goals,
       AVG(home_goal + away_goal) OVER(PARTITION BY season) AS season_avg
FROM match; # считаем среднее по сезону

Параметры в OVER(ROWS BETWEEN):

*   PRECEDING - строка до
*   FOLLOWING - строка после
*   UNBOUNDED PRECEDING - начало
*   UNBOUNDED FOLLOWING - конец набора данных
*   CURRENT ROW - остановить вычисления в текущей строке



In [ ]:
# скользящее окно - вычисления относительно текущей строки
SELECT date, home_goal, away_goal,
       SUM(home_goal) OVER(ORDER BY date ROWS BETWEEN
       UNBOUNDED PRECEDING AND CURRENT ROW) AS running_total
FROM match
WHERE hometeam_id = 8456 AND season = '2011/2012';

SELECT date, home_goal, away_goal, SUM(home_goal)
       OVER(ORDER BY date ROWS BETWEEN 1 PRECEDING AND CURRENT ROW) AS last2
FROM match
WHERE hometeam_id = 8456 AND season = '2011/2012'; # скользящее окно

**PostgreSQL Summary Stats and Window Functions**

In [ ]:
SELECT Year, Event, Country,
       ROW_NUMBER() OVER () AS Row_N
FROM Summer_Medals
WHERE Medal = 'Gold'; # ранжирование просто 1 2 3

SELECT Year, Event, Country,
       ROW_NUMBER() OVER (ORDER BY Year DESC) AS Row_N
FROM Summer_Medals
WHERE Medal = 'Gold'; # даты повторяются, ранги разные уникальные

SELECT Year, Event, Country,
       ROW_NUMBER() OVER(ORDER BY Year DESC, Event ASC) AS Row_N
FROM Summer_Medals
WHERE Medal = 'Gold';

# использование LAG
SELECT Year, Champion,
       LAG(Champion, 1) OVER(ORDER BY Year ASC) AS Last_Champion
FROM Discus_Gold
ORDER BY Year ASC; # столбец с предыдущими значениями

In [ ]:
WITH Discus_Gold AS (...)
SELECT Year, Event, Champion,
       LAG(Champion) OVER(PARTITION BY Event ORDER BY Event ASC, Year ASC) AS Last_Champion
FROM Discus_Gold # что если второй EVENT убрать
ORDER BY Event ASC, Year ASC;
| Year |        Event | Champion | Last_Champion |
|------|--------------|----------|---------------|
| 2004 | Discus Throw |      LTU |          null |
| 2008 | Discus Throw |      EST |           LTU |
| 2012 | Discus Throw |      GER |           EST |
| 2004 |  Triple Jump |      SWE |          null |
| 2008 |  Triple Jump |      POR |           SWE |
| 2012 |  Triple Jump |      USA |           POR |

In [ ]:
LAG() vs LEAD() # вперед
FIRST_VALUE(column) # первое значение в столбце или секции
LAST_VALUE(column) # последнее

LEAD(City, 1) OVER (ORDER BY Year ASC) AS Next_City
# LEAD(Champion, 1) with PARTITION BY Event
# FIRST_VALUE(Champion) with PARTITION BY Event
# наебка с LAST_VALUE
SELECT Year, City, FIRST_VALUE(City) OVER(ORDER BY Year ASC) AS First_City,
LAST_VALUE(City) OVER (ORDER BY Year ASC RANGE BETWEEN
                       UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS Last_City
FROM Hosts
ORDER BY Year ASC; # первое и последнее значение (тут RANGE BETWEEN без него нет)
# расширяется окно

In [ ]:
# ранжирование
# ROW_NUMBER - всегда уникальные индексы
# RANK - однинаковым значениям один индекс с пропуском индекса
# DENSE_RANK - как RANK но без пропуска

ROW_NUMBER() OVER (ORDER BY Games DESC) AS Row_N
RANK() OVER (ORDER BY Games DESC) AS Rank_N
DENSE_RANK() OVER (ORDER BY Games DESC) AS Dense_Rank_N

In [ ]:
# прикольный сценарий
SELECT Country, Athlete, COUNT(*) AS Medals
FROM Summer_Medals
WHERE Country IN ('CHN', 'RUS') AND Year = 2012
GROUP BY Country, Athlete
HAVING COUNT(*) > 1 # это смотри
ORDER BY Country ASC, Medals DESC;

DENSE_RANK() OVER (PARTITION BY Country ORDER BY Medals DESC) AS Rank_N

In [ ]:
# еще одно применение оконных функций - paging (разбиение на страницы)
WITH Disciplines AS (SELECT DISTINCT Discipline FROM Summer_Medals)
SELECT Discipline, NTILE(15) OVER () AS Page
From Disciplines
ORDER BY Page ASC;

|          Discipline | Page |
|---------------------|------|
| Wrestling Freestyle |    1 |
|             Archery |    1 |
|            Baseball |    1 |
|            Lacrosse |    2 |
|                Judo |    2 |
# разеделение на трети и т.д
WITH Country_Medals AS ( SELECT Country, COUNT(*) AS Medals
                         FROM Summer_Medals
                         GROUP BY Country),
SELECT Country, Medals, NTILE(3) OVER (ORDER BY Medals DESC) AS Third
FROM Country_Medals;

In [ ]:
WITH Brazil_Medals AS (...)
SELECT Year, Medals,
       MAX(Medals) OVER (ORDER BY Year ASC) AS Max_Medals #SUM(),
FROM Brazil_Medals;

WITH Medals AS (...)
SELECT Year, Country, Medals,
       SUM(Meals) OVER (PARTITION BY Country ...)
FROM Medals;

In [ ]:
#установка FRAME - изменение поведения оконной функции
WITH Russia_Medals AS (...)

SELECT Year, Medals, MAX(Medals) OVER (ORDER BY Year ASC) AS Max_Medals,
MAX(Medals) OVER (ORDER BY Year ASC ROWS BETWEEN 1 PRECEDING AND CURRENT ROW) AS Max_Medals_Last
MAX(Medals) OVER (ORDER BY Year ASC ROWS BETWEEN CURRENT ROW AND 1 FOLLOWING)
FROM Russia_Medals
ORDER BY Year ASC;

WITH US_Medals AS (...)
SELECT Year, Medals,
AVG(Medals) OVER(ORDER BY Year ASC ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS Medals_MA
FROM US_Medals
ORDER BY Year ASC;

# разница между RANGE BETWEEN и ROWS BETWEEN
# RANGE рассматривает дубликаты в столбцах предположения ORDER BY как отдельные сущности
| Year | Medals | Rows_RT | Range_RT |
|------|--------|---------|----------|
| 1992 |     10 |      10 |       10 |
| 1996 |     50 |      60 |      110 |
| 2000 |     50 |     110 |      110 |
| 2004 |     60 |     170 |      230 |
| 2008 |     60 |     230 |      230 |
| 2012 |     70 |     300 |      300 |

In [ ]:
# преобразование из вертикальной в горизонтальную
SELECT Country, Year, COUNT(*) AS Awards
FROM Summer_Medals
WHERE Country IN ('CHN', 'RUS', 'USA')
      AND Year IN (2008, 2012) AND Medal = 'Gold'
GROUP BY Country, Year
ORDER BY Country ASC, Year ASC;
| Country | Year | Awards |
|---------|------|--------|
|     CHN | 2008 |     74 |
|     CHN | 2012 |     56 |
|     RUS | 2008 |     43 |
|     RUS | 2012 |     47 |
|     USA | 2008 |    125 |
|     USA | 2012 |    147 |

CREATE EXTENSION IF NOT EXISTS tablefunc; # ставим расширение

SELECT * FROM CROSSTAB($$
  SELECT Country, Year, COUNT(*) :: INTEGER AS Awards
  FROM Summer_Medals
  WHERE Country IN ('CHN', 'RUS', 'USA')
        AND Year IN (2008, 2012) AND Medal = 'Gold'
  GROUP BY Country, Year
  ORDER BY Country ASC, Year ASC;
$$) AS ct (Country VARCHAR, "2008" INTEGER, "2012" INTEGER)
ORDER BY Country ASC;

| Country | 2008 | 2012 |
|---------|------|------|
|     CHN |   74 |   56 |
|     RUS |   43 |   47 |
|     USA |  125 |  147 |

In [ ]:
WITH Country_Awards AS (SELECT Country, Year, COUNT(*) AS Awards
                        FROM Summer_Medals
                        WHERE Country IN ('CHN', 'RUS', 'USA')
                              AND Year IN (2004, 2008, 2012)
                              AND Medal = 'Gold' AND Sport = 'Gymnastics'
                        GROUP BY Country, Year
                        ORDER BY Country ASC, Year ASC)
SELECT Country, Year,
RANK() OVER (PARTITION BY Year ORDER BY Awards DESC) :: INTEGER AS rank
FROM Country_Awards
ORDER BY Country ASC, Year ASC;
| Country | Year |Rank |
|---------|------|-----|
|     CHN | 2004 |   3 |
|     CHN | 2008 |   1 |
|     CHN | 2012 |   1 |
|     RUS | 2004 |   1 |
|     RUS | 2008 |   2 |

CREATE EXTENSION IF NOT EXISTS tablefunc;
SELECT * FROM CROSSTAB($$...$$) AS ct (
    Country VARCHAR, "2004" INTEGER, "2008" INTEGER, "2012" INTEGER)
ORDER BY Country ASC;

| Country | 2004 | 2008 | 2012 |
|---------|------|------|------|
|     CHN |    3 |    1 |    1 |
|     RUS |    1 |    2 |    2 |
|     USA |    2 |    3 |    3 |

In [ ]:
# ROLLUP и CUBE вычисление промежуточных итогов (групповые и общие)
SELECT Country, Medal, COUNT(*) AS Awards
FROM Summer_Medals
WHERE Year = 2008 AND Country IN ('CHN', 'RUS')
GROUP BY Country, ROLLUP(Medal) # добавка
ORDER BY Country ASC, Medal ASC;
# итоги на уровне страны и медали, потом только на страны, а медали NULL

# GROUP BY ROLLUP(Country, Medal) будут итоги на уровне страны
# ROLLUP(Medal, Country) будут итоги на уровне медали

SELECT Country, Medal, COUNT(*) AS Awards
FROM summer_medals
WHERE Year = 2008 AND Country IN ('CHN', 'RUS')
GROUP BY CUBE(Country, Medal)
ORDER BY Country ASC, Medal ASC; # подсчет всего

SELECT COALESCE(Country, 'Both countries') AS Country,
       COALESCE(Medal, 'All medals') AS Medal,
       COUNT(*) AS Awards
FROM summer_medals
WHERE Year = 2008 AND Country IN ('CHN', 'RUS')
GROUP BY ROLLUP(Country, Medal)
ORDER BY Country ASC, Medal ASC;

STRING_AGG(Letter, ',') # зачем она значения столбцов в строку с разделителем

**Functions for Manipulating Data in PostgreSQL**

In [ ]:
# инфа о базах данных в системной базе данных
SELECT column_name, data_type
FROM INFORMATION_SCHEMA.COLUMNS
WHERE column_name in ('title', 'description', 'special_features')
AND table_name ='film'; # тип столбцов

In [ ]:
CREATE TABLE my_first_table (first_column text,
                             second_column integer);
# помещение данных
INSERT INTO my_first_table
      (first_column, second_column) VALUES ('text value', 12);

In [ ]:
# создаем массив и вложенный массив
CREATE TABLE grades (student_id int,
                     email text[][],
                     test_scores int[]);
INSERT INTO grades
       VALUES (1,
       '{{"work", "work1@datacamp.com"}, {"other", "other1@datacamp.com"}}',
       '{92,85,96,88}' ); # суть в кавычках ' '

# доступ
SELECT email[1][1] AS type, email[1][2] AS address, test_scores[1],
FROM grades
WHERE email[1][1] = 'work';

# WHERE 'other' = ANY (email); # ищет в массиве 'other' и возврат строки инчае null
#альтернатива (contains): WHERE email @> ARRAY['other'];

In [ ]:
SELECT date '2005-09-11' - date '2005-09-10'; # 1 целое
SELECT date '2005-09-11' + integer '3'; # '2005-09-14'
SELECT date '2005-09-11 00:00:00'-date '2005-09-09 12:00:00'; # интервал 1 day 12:00:00
SELECT AGE(timestamp '2005-09-11 00:00:00', timestamp '2005-09-09 12:00:00');
# операция для метки времени и получаем интервал
SELECT rental_date + INTERVAL '3 days' as expected_return
FROM rental; # 2005-05-27 22:53:30

SELECT timestamp '2019-05-01' + 21 * INTERVAL '1 day'; #  2019-05-22 00:00:00

In [ ]:
SELECT NOW(); # текущая метка времени с поясом
SELECT NOW()::timestamp; # преобразование без пояса (для PosrgreSQL)
SELECT CAST(NOW() as timestamp); # аналогично
SELECT CURRENT_TIMESTAMP(2); # для PostgreSQL но с контролем точности
SELECT CURRENT_DATE;
SELECT CURRENT_TIME;

# извлечение информации
SELECT EXTRACT(quarter FROM timestamp '2005-01-24 05:12:00') AS quarter;
SELECT DATE_PART('quarter', timestamp '2005-01-24 05:12:00') AS quarter;
# отнесет метку времени (времени и интервала) к кварталу

SELECT EXTRACT(quarter FROM payment_date) AS quarter,
       EXTRACT(year FROM payment_date) AS year,
       SUM(amount) AS total_payments
FROM payment
GROUP BY 1, 2; # ааааааааааа

SELECT DATE_TRUNC('year', TIMESTAMP '2005-05-21 15:30:30');
# Result: 2005-01-01 00:00:00 (усечение до года) выход метка или интервал

SELECT DATE_TRUNC('month', TIMESTAMP '2005-05-21 15:30:30');
#Result: 2005-05-01 00:00:00 (усечение до месяца) выход метка или интервал

In [ ]:
# объединение строк
SELECT first_name, last_name,
       first_name || ' ' || last_name AS full_name
FROM customer
# аналогично
SELECT CONCAT(first_name, ' ', last_name) AS full_name
FROM customer;

SELECT customer_id || ': ' || first_name || ' ' || last_name AS full_name
FROM customer;

SELECT
UPPER(email), LOWER(title), INITCAP(title)#первые буквы большие даже после пробела
FROM customer;

SELECT # заменить в столбце это на это
REPLACE(description, 'A Astounding', 'An Astounding') as description
FROM film;

SELECT title, REVERSE(title) # реверс слов
FROM film AS f;

SELECT CHAR_LENGTH(name), LENGTH(name) # длина текста
SELECT POSITION('@' IN email), STRPOS(email, '@')

SELECT LEFT(description, 50), RIGHT(description, 50) # первые и последние n слов
SELECT SUBSTRING(description, 10, 50)
SELECT SUBSTRING(email FROM 0 FOR POSITION('@' IN email))

TRIM([leading | trailing | both] [characters] from string)
# параметры: удаление в начале или конце (обычно везде)
# символ который удалется
# строка
SELECT TRIM(' padded ');
SELECT LTRIM(' padded '), RTRIM(' padded '); # удаляем из начала и конца

SELECT LPAD('padded', 10),  LPAD('padded', 5); # добавление пробелов слева и справа
SELECT RPAD('padded', 10, '#');

In [ ]:
SELECT title
FROM film
WHERE title LIKE 'ELF%';
# Полнотекстовый поиск
SELECT title, description
FROM film
WHERE to_tsvector(title) @@ to_tsquery('elf');
# процедура не чувствительна к регистру, есть ли в предложении elf

Введение в расширения PostgreSQL

In [ ]:
# определяемые пользователем типы данных (регистрирует тип в системной таблице)
CREATE TYPE dayofweek AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday',
                               'Friday', 'Saturday', 'Sunday');
SELECT typname, typcategory
FROM pg_type # системная таблица
WHERE typname='dayofweek';
+-----------+-------------+
|   typname | typcategory |
|-----------|-------------|
| dayofweek |           E | #тип данных Е ENUM
+-----------+-------------+

SELECT column_name, data_type, udt_name #udt_name значение имени при создании CREATE TYPE
FROM INFORMATION_SCHEMA.COLUMNS # системная таблица
WHERE table_name ='film';
+-----------------------------------------------+
| column_name |         data_type |    udt_name |
|-------------|-------------------|-------------|
|       title | character varying |     varchar |
|      rating |      USER-DEFINED | mpaa_rating |
+-----------------------------------------------+

In [ ]:
# пользовательские функции
CREATE FUNCTION squared(i integer) RETURNS integer AS $$
        BEGIN
            RETURN i * i;
        END;
$$ LANGUAGE plpgsql;
SELECT squared(10);

In [ ]:
SELECT name # Available Extensions
FROM pg_available_extensions;

SELECT extname # Installed Extensions
FROM pg_extension;

# загрузка расширений в базу данных и включение
CREATE EXTENSION IF NOT EXISTS fuzzystrmatch;
SELECT extname FROM pg_extension;
# функция из расширения
SELECT levenshtein('GUMBO', 'GAMBOL');
SELECT similarity('GUMBO', 'GAMBOL'); # сходство на триграммах

**Exploratory Data Analysis in SQL**

In [ ]:
count(DISTINCT column_name) # не включает null
DISTINCT column_name # включает null
count(*) # включает

# есть внешние включи которые ссылаются к другой таблице
# и первичные ключи

SELECT coalesce(column_1, column_2)
FROM prices; # возврат значения отличного от null
 column_1 | column_2      coalesce
----------+----------    ---------
          | 10               10
          |
       22 |                  22
        3 | 4                 3

In [ ]:
# изменение типа
SELECT CAST (3.7 AS integer); # 4
SELECT CAST (total AS integer)
FROM prices;
Или SELECT 3.7 :: integer
# Практика. Group by name :: int аааааааааааааааааааааааааааааааааааааааааааа

In [ ]:
SELECT var_pop(question_pct) # дисперсия теоретическая
SELECT var_samp(question_pct), variance(question_pct) # выборки
SELECT stddev_samp(question_pct),  stddev_pop(question_pct), stddev(question_pct)

SELECT trunc(42.1256, 2),  trunc(12345, -3); # 42.12 и 12000

SELECT trunc(unanswered_count, -1) AS trunc_ua, count(*)
FROM stackoverflow
WHERE tag='amazon-ebs'
GROUP BY trunc_ua
ORDER BY trunc_ua; # можно и так аааааааааааааааааааааааааааааааааааааааааааааааа

SELECT generate_series(0, 1, .1); # от до шаг

In [ ]:
# пример подсчета сколько попадает между интервалами
WITH bins AS (SELECT generate_series(30,60,5) AS lower,
                     generate_series(35,65,5) AS upper),
     ebs AS (SELECT unanswered_count
             FROM stackoverflow
             WHERE tag='amazon-ebs')

SELECT lower, upper, count(unanswered_count)
FROM bins
LEFT JOIN ebs
ON unanswered_count >= lower AND unanswered_count < upper
GROUP BY lower, upper
ORDER BY lower;

In [ ]:
SELECT corr(assets, equity) # корреляция
FROM fortune500;
# медиана, важно упорядочить
SELECT percentile_disc(.5) WITHIN GROUP (ORDER BY name) # выдаст что есть
SELECT percentile_cont(.5) WITHIN GROUP (ORDER BY name) # придумает


In [ ]:
# создание временных таблиц (способ первый)
CREATE TEMP TABLE new_tablename AS
       SELECT column1, column2
       FROM table;
# создание временных таблиц (способ второй PostgreSQL)
SELECT column1, column2
INTO TEMP TABLE new_tablename
FROM table;

#добавление строк в временную таблицу
INSERT INTO top_companies
SELECT rank, title # столбцы должны совпадать
FROM fortune500
WHERE rank BETWEEN 11 AND 20;

# удаление два способа
DROP TABLE top_companies;
DROP TABLE IF EXISTS top_companies;

Техt данные

In [ ]:
'apple' != 'Apple'
'' != NULL
' apple' != 'apple'
'' != ' '
'to-do' != 'to–do' # пунктуация
#Практика. При group by null подсчитывается, даже если count(id)
SELECT lower('aBc DeFg 7-'), upper('aBc DeFg 7-');
# удаляет пробелы
SELECT trim(' abc '), rtrim(' abc '), ltrim(' abc ')
SELECT trim('Wow!', '!'), trim('Wow!', '!wW') #выход 'o'

#взять только n в начале или конце
SELECT left('abcde', 2), right('abcde', 2);
SELECT substring('abcdef' FROM 2 FOR 3);
SELECT substr('abcdef', 2, 3);

SELECT split_part('a,bc,d', ',', 2); # откуда, как разбить, какую позицию взять
SELECT split_part('cats and dogs and fish', ' and ', 1); # cats

SELECT concat('a', 2, 'cc'), 'a' || 2 || 'cc'; # a2cc
SELECT concat('a', NULL,'cc'),  'a' || NULL || 'cc'; # acc и ничего

In [ ]:
     id |                                    category | businesses
--------+---------------------------------------------+------------
 111110 |                Agriculture: Soybean Farming | 4788
 111130 |      Agriculture | Dry Pea and Bean Farming | 3606
 111140 |                  Agriculture: Wheat Farming | 6393

SELECT CASE WHEN category LIKE '%: %' THEN split_part(category, ': ', 1)
            WHEN category LIKE '% - %' THEN split_part(category, ' - ', 1)
            ELSE split_part(category, ' | ', 1)
      END AS major_category, # ты понимаешь о чем ааааааааааааааааааа
      sum(businesses)
FROM naics
GROUP BY major_category; # можно

In [ ]:
CREATE TEMP TABLE recode AS
       SELECT DISTINCT fav_fruit AS original, fav_fruit AS standardized
       FROM fruit; # DISTINCT по обоим столбцам
 original | standardized
----------+--------------
   APPLES | APPLES
    apple | apple
    Apple | Apple
  bannana | bannana

# перезаписать столбец
UPDATE recode
SET standardized=trim(lower(original));

UPDATE recode
SET standardized='banana'
WHERE standardized LIKE '%nn%'; # для строк с nn

UPDATE recode
SET standardized=rtrim(standardized, 's'); # удаляем s в конце

SELECT standardized, count(*)
FROM fruit
LEFT JOIN recode ON fav_fruit=original
GROUP BY standardized;

Дата и время

In [ ]:
# date YYYY-MM-DD
# timestamp YYYY-MM-DD HH:MM:SS (тут еще пояс бывает)
SELECT '2018-01-01' > '2017-12-31';
SELECT now() - '2015-01-01'; # 1439 days 21:32:22.616076

SELECT '2010-01-01'::date + 1; # 2010-01-02 добавит дни
# добавление целого числа к метке времени приведет к ошибке
SELECT '2018-12-10'::date + '1 year'::interval; # 2019-12-10 00:00:00
SELECT '2018-12-10'::date + '1 year 2 days 3 minutes'::interval;#2019-12-12 00:03:00

In [ ]:
SELECT date_part('month', now()), EXTRACT(MONTH FROM now());
# извлечение номер месяца
SELECT date_part('month', date) AS month, sum(amt)
FROM sales
GROUP BY month
ORDER BY month;

-- now() is 2018-12-17 21:45:15.6829-06
SELECT date_trunc('month', now()); # 2018-12-01 00:00:00-06

SELECT generate_series('2018-01-01', '2018-01-15', '2 days'::interval);
# генерирование интервала
SELECT generate_series('2018-01-01', '2018-01-02', '5 hours'::interval);
SELECT generate_series('2018-01-31', '2018-12-31', '1 month'::interval);

# генерировать на конец месяца сложно так как 31 на 28 и дальше 28 28 28 поэтому
SELECT generate_series('2018-02-01','2019-01-01', '1 month'::interval) - '1 day'::interval;

In [ ]:
# добавить часы где не было продаж
WITH hour_series AS (
SELECT generate_series('2018-04-23 09:00:00', '2018-04-23 14:00:00', '1 hour'::interval) AS hours)

SELECT hours, count(date)
FROM hour_series
    LEFT JOIN sales
        ON hours=date_trunc('hour', date)
GROUP BY hours
ORDER BY hours;

In [ ]:
# еще пример
WITH bins AS (SELECT generate_series('2018-04-23 09:00:00', '2018-04-23 15:00:00','3 hours'::interval) AS lower,
                     generate_series('2018-04-23 12:00:00','2018-04-23 18:00:00','3 hours'::interval) AS upper)

SELECT lower, upper, count(date)
FROM bins
     LEFT JOIN sales
            ON date >= lower
           AND date < upper
GROUP BY lower, upper
ORDER BY lower;

In [ ]:
SELECT date, lag(date) OVER (ORDER BY date),
             lead(date) OVER (ORDER BY date)
FROM sales;

SELECT date,
date - lag(date) OVER (ORDER BY date) AS gap
FROM sales;

                date | gap
---------------------+----------
 2018-04-23 09:07:33 |
 2018-04-23 09:13:14 | 00:05:41
 2018-04-23 09:35:16 | 00:22:02
 2018-04-23 10:12:35 | 00:37:19

SELECT avg(gap)
FROM (SELECT date - lag(date) OVER (ORDER BY date) AS gap
FROM sales) AS gaps;
       avg
-----------------
00:32:15.555556

SELECT date, amount,
lag(amount) OVER (ORDER BY date), amount - lag(amount) OVER (ORDER BY date) AS change
FROM sales;

**Data-Driven Decision Making in SQL**

In [ ]:
SELECT *
FROM movies as m
WHERE 5 >
    (SELECT COUNT(*)
     FROM renting as r
     WHERE r.movie_id = m.movie_id);

In [ ]:
# TRUE возвращается, если результат
# коррелированного вложенного запроса содержит хотя бы одну строку
SELECT *
FROM movies AS m
WHERE EXISTS
     (SELECT *
      FROM renting AS r
      WHERE rating IS NOT NULL
      AND r.movie_id = m.movie_id);

SELECT *
FROM movies AS m
WHERE NOT EXISTS # верент True если пусто
      (SELECT *
       FROM renting AS r
       WHERE rating IS NOT NULL
       AND r.movie_id = m.movie_id);

In [ ]:
# UNION (без дубликатов)
SELECT title, genre, renting_price
FROM movies
WHERE renting_price > 2.8
UNION
SELECT title, genre, renting_price
FROM movies
WHERE genre = 'Action & Adventure';

# INTERSECT (пересечение)
SELECT title, genre, renting_price
FROM movies
WHERE renting_price > 2.8
INTERSECT
SELECT title, genre, renting_price
FROM movies
WHERE genre = 'Action & Adventure';

In [ ]:
# CUBE, ROLLUP, GROUPING SETS
SELECT country, genre, COUNT(*) # длинный формат
FROM renting_extended
GROUP BY CUBE (country, genre);

|  country |  genre | count |
|----------|--------|-------|
|  Austria | Comedy |     2 |
|  Belgium |  Drama |    15 |
|  Austria |  Drama |     4 |
|  Belgium | Comedy |     1 |
|  Belgium |   null |    16 |
|  Austria |   null |     6 |
|     null | Comedy |     3 |
|     null |  Drama |    19 |
|     null |   null |    22 |

SELECT country, genre, COUNT(*)
FROM renting_extended
GROUP BY ROLLUP (country, genre);  GROUP BY ROLLUP (genre, country);

| country |  genre | count |   | country |  genre | count |
|---------|--------|-------|   |---------|--------|-------|
|    null |   null |    22 |   |    null |   null |    22 |
| Austria | Comedy |     2 |   | Austria | Comedy |     2 |
| Belgium |  Drama |    15 |   | Belgium |  Drama |    15 |
| Austria |  Drama |     4 |   | Austria |  Drama |     4 |
| Belgium | Comedy |     1 |   | Belgium | Comedy |     1 |
| Belgium |   null |    16 |   |    null | Comedy |     3 |
| Austria |   null |     6 |   |    null |  Drama |    19 |

SELECT country, genre, COUNT(*) # выбор уровней агрегации
FROM rentings_extended
GROUP BY GROUPING SETS ((country, genre), (country), (genre), ());

| country |  genre |count|
|---------|--------|-----|
|    NULL |   NULL |  22 |
| Austria | Comedy |   2 |
| Belgium |  Drama |  15 |
| Austria |  Drama |   4 |
| Belgium | Comedy |   1 |
| Belgium |   NULL |  16 |
| Austria |   NULL |   6 |
|    NULL | Comedy |   3 |
|    NULL |  Drama |  19 |